In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import subspace_modified

In [2]:
# Sensor Definition
lowersensor_y = np.arange(1,116,6)
lowersensor_y_less6 = np.arange(13,116,18)
lowersensor_y_less10 = np.arange(1,116,12)

Middlesensor_y = np.arange(3,118,6)
Middlesensor_y_less6 = np.arange(15,118,18)
Middlesensor_y_less10 = np.arange(3,118,12)

Uppersensor_y = np.arange(5,120,6)
Uppersensor_y_less6 = np.arange(17,120,18)
Uppersensor_y_less10 = np.arange(5,120,12)

# Prepare the modes data
    # modes_Modal-->modes_Modal_Selected
    # acceleration -->modes_SSI --> modes_normalised(modes_SSI2, modes_SSI4)

In [3]:
modesModalPathdic={ \
0:'modes_Modal\\undamaged_modal_modes.dat', \
1:'modes_Modal\\damaged7_20%_modal_modes.dat', \
2:'modes_Modal\\damaged7_50%_modal_modes.dat', \
3:'modes_Modal\\damaged7_70%_modal_modes.dat'}
modesModalSelectedPathdic_Uppersensor_y = { \
0:'modes_Modal_Selected\\undamaged_Uppersensor_y.dat', \
1:'modes_Modal_Selected\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_Modal_Selected\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_Modal_Selected\\damaged7_70%_Uppersensor_y.dat'}        
modesModalSelectedPathdic_Uppersensor_y_less10 = { \
0:'modes_Modal_Selected\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_Modal_Selected\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_Modal_Selected\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_Modal_Selected\\damaged7_70%_Uppersensor_y_less10.dat'}  
modesModalSelecteddic_Uppersensor_y = {}
modesModalSelecteddic_Uppersensor_y_less10 = {}

In [4]:
class Modesdata_Modal:
    def __init__(self,modesrealpath,modes_Modal_selectedPath,sensor = np.arange(3,118,6)):
        self.modesrealPath = modesrealpath
        self.sensor = sensor
        self.sensornumber = self.sensor.size
        self.modes_Modal_selectedPath = modes_Modal_selectedPath
        # load modes_Modal data
        self.modes_Modal = np.loadtxt(self.modesrealPath)
        # Sensor definition lowersensor_y = np.arange(1,116,6) Middlesensor_y = np.arange(3,118,6)Uppersensor_y = np.arange(5,120,6)
        self.modes_Modal_selected = self.modes_Modal[self.sensor].T   
        # save the modes_Modal_selected
        f = open(self.modes_Modal_selectedPath,'w')
        np.savetxt(f,self.modes_Modal_selected,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.modes_Modal_selectedPath))
        f.close()

In [5]:
for i in range(4):
    modesModalSelecteddic_Uppersensor_y[i]=Modesdata_Modal(modesModalPathdic[i],modesModalSelectedPathdic_Uppersensor_y[i],Uppersensor_y)
    modesModalSelecteddic_Uppersensor_y_less10[i]=Modesdata_Modal(modesModalPathdic[i],modesModalSelectedPathdic_Uppersensor_y_less10[i],Uppersensor_y_less10)

  modes_Modal_Selected\undamaged_Uppersensor_y.dat written Finished.
  modes_Modal_Selected\undamaged_Uppersensor_y_less10.dat written Finished.
  modes_Modal_Selected\damaged7_20%_Uppersensor_y.dat written Finished.
  modes_Modal_Selected\damaged7_20%_Uppersensor_y_less10.dat written Finished.
  modes_Modal_Selected\damaged7_50%_Uppersensor_y.dat written Finished.
  modes_Modal_Selected\damaged7_50%_Uppersensor_y_less10.dat written Finished.
  modes_Modal_Selected\damaged7_70%_Uppersensor_y.dat written Finished.
  modes_Modal_Selected\damaged7_70%_Uppersensor_y_less10.dat written Finished.


## accelerations -->modes_SSI --> modes_normalised(modes_SSI2, modes_SSI4)

In [6]:
class Accdata:   
    def __init__(self,accpath,modespath,modesRealpath,modesfreqspath,mino = 16,maxo = 18,noiselevel=0,sensor=np.arange(3,118,6)):
        self.accpath = accpath
        self.modesFullPath = modespath
        self.modesRealPath = modesRealpath
        self.modesfreqspath = modesfreqspath
        self.mino = mino
        self.maxo = maxo
        self.noiselevel = noiselevel
        self.sensor = sensor
    def accTomodes(self):
        # load acceleration data
        output = np.loadtxt(self.accpath)
        output_noise = subspace_modified.addNoise(output.T,self.noiselevel).T
        # Sensor definition
        data = output_noise[:,self.sensor] # Choose all the sensors for ay along the middleline
        # Sensor definition Lowersensor_y = np.arange(1,116,6) Middlesensor_y = np.arange(3,118,6) Uppersensor_y = np.arange(5,120,6)
        SSI = subspace_modified.SSI(data.T, 0.005, rows = 20, method = 'cov')
        # SSI.plotSingularValues()
        self.freqs, self.dampings, self.modes = SSI.getStableModalProperties(mino=self.mino, maxo=self.maxo, minf=0, maxf=40, ftol=1e-3, mtol=1e-3)
        self.Real = np.real(self.modes[2])
        self.modes = np.array(self.modes)
        # save the 3d array modes
        self.modes.tofile(self.modesFullPath,sep=',',format='%s')
        pipe=sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.modesFullPath))
        # save the real part of eigenvectors
        f = open(self.modesRealPath,'w')
        np.savetxt(f,self.Real,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.modesRealPath))
        f.close()
        # save the freqs
        f = open(self.modesfreqspath,'w')
        np.savetxt(f,self.freqs,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.modesfreqspath))
        f.close()

In [7]:
accPathdic = \
 {0:'accelerations\\undamaged_accelerations.dat', \
1:'accelerations\\damaged7_20%_accelerations.dat', \
2:'accelerations\\damaged7_50%_accelerations.dat', \
3:'accelerations\\damaged7_70%_accelerations.dat'}
#Uppersensor_y
modesFullPathdic_Uppersensor_y =  \
{0:'modes_SSI_order18\\undamaged_modes_Uppersensor_y.dat', \
1:'modes_SSI_order18\\damaged7_20%_modes_Uppersensor_y.dat', \
2:'modes_SSI_order18\\damaged7_50%_modes_Uppersensor_y.dat', \
3:'modes_SSI_order18\\damaged7_70%_modes_Uppersensor_y.dat'}
modesRealPathdic_Uppersensor_y =  \
{0:'modes_SSI_order18\\undamaged_modes_real_Uppersensor_y.dat', \
1:'modes_SSI_order18\\damaged7_20%_modes_real_Uppersensor_y.dat', \
2:'modes_SSI_order18\\damaged7_50%_modes_real_Uppersensor_y.dat', \
3:'modes_SSI_order18\\damaged7_70%_modes_real_Uppersensor_y.dat'}
freqsPathdic_Uppersensor_y =  \
{0:'modes_SSI_order18\\undamaged_freqs_Uppersensor_y.dat', \
1:'modes_SSI_order18\\damaged7_20%_freqs_Uppersensor_y.dat', \
2:'modes_SSI_order18\\damaged7_50%_freqs_Uppersensor_y.dat', \
3:'modes_SSI_order18\\damaged7_70%_freqs_Uppersensor_y.dat'}
# Uppersensor_y_less10
modesFullPathdic_Uppersensor_y_less10 =  \
{0:'modes_SSI_order18\\undamaged_modes_Uppersensor_y_less10.dat', \
1:'modes_SSI_order18\\damaged7_20%_modes_Uppersensor_y_less10.dat', \
2:'modes_SSI_order18\\damaged7_50%_modes_Uppersensor_y_less10.dat', \
3:'modes_SSI_order18\\damaged7_70%_modes_Uppersensor_y_less10.dat'}
modesRealPathdic_Uppersensor_y_less10 =  \
{0:'modes_SSI_order18\\undamaged_modes_real_Uppersensor_y_less10.dat', \
1:'modes_SSI_order18\\damaged7_20%_modes_real_Uppersensor_y_less10.dat', \
2:'modes_SSI_order18\\damaged7_50%_modes_real_Uppersensor_y_less10.dat', \
3:'modes_SSI_order18\\damaged7_70%_modes_real_Uppersensor_y_less10.dat'}
freqsPathdic_Uppersensor_y_less10 =  \
{0:'modes_SSI_order18\\undamaged_freqs_Uppersensor_y_less10.dat', \
1:'modes_SSI_order18\\damaged7_20%_freqs_Uppersensor_y_less10.dat', \
2:'modes_SSI_order18\\damaged7_50%_freqs_Uppersensor_y_less10.dat', \
3:'modes_SSI_order18\\damaged7_70%_freqs_Uppersensor_y_less10.dat'}
accCasedic_Uppersensor_y = {}
accCasedic_Uppersensor_y_less10 = {}
# Write the modes_Uppersensor_y 
for i in range(4):
    accPath = accPathdic[i]
    modesFullPath = modesFullPathdic_Uppersensor_y[i]
    modesRealPath = modesRealPathdic_Uppersensor_y[i]
    freqsPath = freqsPathdic_Uppersensor_y [i]
    accCasedic_Uppersensor_y[i] = Accdata(accPath,modesFullPath,modesRealPath,freqsPath,mino=16,maxo=18,sensor=Uppersensor_y)
    accCasedic_Uppersensor_y[i].accTomodes()
# Write the modes_Uppersensor_y_less10
for i in range(4):
    accPath = accPathdic[i]
    modesFullPath = modesFullPathdic_Uppersensor_y_less10[i]
    modesRealPath = modesRealPathdic_Uppersensor_y_less10[i]
    freqsPath = freqsPathdic_Uppersensor_y_less10 [i]
    accCasedic_Uppersensor_y_less10[i] = Accdata(accPath,modesFullPath,modesRealPath,freqsPath,mino=16,maxo=18,sensor=Uppersensor_y_less10)
    accCasedic_Uppersensor_y_less10[i].accTomodes()

Stochastic Subspace Identification (SSI) - Cov
  forming block Hankel matrix...
  performing singular value decomposition...
  processing completed.
Stochastic Subspace Identification (SSI) - Cov
  calculating stable modal properties...
  processing completed.
  modes_SSI_order18\undamaged_modes_Uppersensor_y.dat written Finished.
  modes_SSI_order18\undamaged_modes_real_Uppersensor_y.dat written Finished.
  modes_SSI_order18\undamaged_freqs_Uppersensor_y.dat written Finished.
Stochastic Subspace Identification (SSI) - Cov
  forming block Hankel matrix...
  performing singular value decomposition...
  processing completed.
Stochastic Subspace Identification (SSI) - Cov
  calculating stable modal properties...
  processing completed.
  modes_SSI_order18\damaged7_20%_modes_Uppersensor_y.dat written Finished.
  modes_SSI_order18\damaged7_20%_modes_real_Uppersensor_y.dat written Finished.
  modes_SSI_order18\damaged7_20%_freqs_Uppersensor_y.dat written Finished.
Stochastic Subspace Identif

In [10]:
class Modesdata_SSI:
    def __init__(self,modesSSIrealpath,modesSSI2Path,modesSSI4Path):
        self.modesSSIrealPath = modesSSIrealpath
        self.modesSSI2Path = modesSSI2Path
        self.modesSSI4Path = modesSSI4Path
        
    def normalisation(self):
        # load modes_SSI data
        self.modes_SSI = np.loadtxt(self.modesSSIrealPath,delimiter = ',')
        self.order,self.sensornumber = self.modes_SSI.shape
        # Normalisation SSI2  # Normalisation SSI1 == SSI2
        # modes_SSI1 = modes_SSI.T/(np.max(np.abs(modes_SSI.T),0))
        # modes_SSI1 = modes_SSI1.T
        self.modes_SSI2 = np.zeros_like(self.modes_SSI)
        for i in range(self.order):
            self.modes_SSI2[i] = self.modes_SSI[i]/np.max(abs(self.modes_SSI[i])) 
        # save the modes_SSI2
        f = open(self.modesSSI2Path,'w')
        np.savetxt(f,self.modes_SSI2,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.modesSSI2Path))
        f.close()
        # Normalisation SSI4
        self.modes_SSI4 = np.zeros_like(self.modes_SSI)
        for i in range(self.order):
            self.modes_SSI4[i] = self.modes_SSI[i]/np.linalg.norm(self.modes_SSI[i]) 
        # save the modes_SSI4
        f = open(self.modesSSI4Path,'w')
        np.savetxt(f,self.modes_SSI4,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.modesSSI4Path))
        f.close()
    def SaveFrequency(self,freqsPath,frequenciesPath):
        self.freqsPath  = freqsPath
        self.frequenciesPath = frequenciesPath
        self.freq = np.loadtxt(self.freqsPath, delimiter=',')
        self.frequency = self.freq[0]
        # save the frequency_SSI
        f = open(self.frequenciesPath,'w')
        np.savetxt(f,self.frequency,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.frequenciesPath))
        f.close()        

In [12]:
modesSSI2Pathdic_Uppersensor_y = \
{0:'modes_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'modes_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
modesSSI2Pathdic_Uppersensor_y_less10 = \
{0:'modes_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
modesSSI4Pathdic_Uppersensor_y = \
{0:'modes_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'modes_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
modesSSI4Pathdic_Uppersensor_y_less10 = \
{0:'modes_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
freqsPathdic_Uppersensor_y =  \
{0:'modes_SSI_order18\\undamaged_freqs_Uppersensor_y.dat', \
1:'modes_SSI_order18\\damaged7_20%_freqs_Uppersensor_y.dat', \
2:'modes_SSI_order18\\damaged7_50%_freqs_Uppersensor_y.dat', \
3:'modes_SSI_order18\\damaged7_70%_freqs_Uppersensor_y.dat'}
freqsPathdic_Uppersensor_y_less10 =  \
{0:'modes_SSI_order18\\undamaged_freqs_Uppersensor_y_less10.dat', \
1:'modes_SSI_order18\\damaged7_20%_freqs_Uppersensor_y_less10.dat', \
2:'modes_SSI_order18\\damaged7_50%_freqs_Uppersensor_y_less10.dat', \
3:'modes_SSI_order18\\damaged7_70%_freqs_Uppersensor_y_less10.dat'}
frequencyPathdic_Uppersensor_y =  \
{0:'modes_SSI_order18\\undamaged_frequency_Uppersensor_y.dat', \
1:'modes_SSI_order18\\damaged7_20%_frequency_Uppersensor_y.dat', \
2:'modes_SSI_order18\\damaged7_50%_frequency_Uppersensor_y.dat', \
3:'modes_SSI_order18\\damaged7_70%_frequency_Uppersensor_y.dat'}
frequencyPathdic_Uppersensor_y_less10 =  \
{0:'modes_SSI_order18\\undamaged_frequency_Uppersensor_y_less10.dat', \
1:'modes_SSI_order18\\damaged7_20%_frequency_Uppersensor_y_less10.dat', \
2:'modes_SSI_order18\\damaged7_50%_frequency_Uppersensor_y_less10.dat', \
3:'modes_SSI_order18\\damaged7_70%_frequency_Uppersensor_y_less10.dat'}

modesSSIcasedic_Uppersensor_y = {}
modesSSIcasedic_Uppersensor_y_less10 = {}
for i in range(4):
    modesSSIcasedic_Uppersensor_y[i] = Modesdata_SSI(modesRealPathdic_Uppersensor_y[i], \
                                       modesSSI2Pathdic_Uppersensor_y[i], \
                                       modesSSI4Pathdic_Uppersensor_y[i])
    modesSSIcasedic_Uppersensor_y[i].normalisation()
    modesSSIcasedic_Uppersensor_y[i].SaveFrequency(freqsPathdic_Uppersensor_y[i],frequencyPathdic_Uppersensor_y[i])
for i in range(4):
    modesSSIcasedic_Uppersensor_y_less10[i] = Modesdata_SSI(modesRealPathdic_Uppersensor_y_less10[i], \
                                       modesSSI2Pathdic_Uppersensor_y_less10[i], \
                                       modesSSI4Pathdic_Uppersensor_y_less10[i])
    modesSSIcasedic_Uppersensor_y_less10[i].normalisation()
    modesSSIcasedic_Uppersensor_y_less10[i].SaveFrequency(freqsPathdic_Uppersensor_y_less10[i],frequencyPathdic_Uppersensor_y_less10[i])

  modes_SSI2_order18\undamaged_Uppersensor_y.dat written Finished.
  modes_SSI4_order18\undamaged_Uppersensor_y.dat written Finished.
  modes_SSI_order18\undamaged_frequency_Uppersensor_y.dat written Finished.
  modes_SSI2_order18\damaged7_20%_Uppersensor_y.dat written Finished.
  modes_SSI4_order18\damaged7_20%_Uppersensor_y.dat written Finished.
  modes_SSI_order18\damaged7_20%_frequency_Uppersensor_y.dat written Finished.
  modes_SSI2_order18\damaged7_50%_Uppersensor_y.dat written Finished.
  modes_SSI4_order18\damaged7_50%_Uppersensor_y.dat written Finished.
  modes_SSI_order18\damaged7_50%_frequency_Uppersensor_y.dat written Finished.
  modes_SSI2_order18\damaged7_70%_Uppersensor_y.dat written Finished.
  modes_SSI4_order18\damaged7_70%_Uppersensor_y.dat written Finished.
  modes_SSI_order18\damaged7_70%_frequency_Uppersensor_y.dat written Finished.
  modes_SSI2_order18\undamaged_Uppersensor_y_less10.dat written Finished.
  modes_SSI4_order18\undamaged_Uppersensor_y_less10.dat wri